<h1 style="font-family: 'Times New Roman'; text-align: center; font-weight: bold;">
    CoxL BLAST Output Preprocessing
</h1>

<div style="font-family: 'Times New Roman'; font-size: 15px; text-align: justify; width: 100%;">
  <div>
    <span style="display: inline-block; width: 100px;"><b>Date</b></span>: 15<sup>th</sup> November 2024
  </div>
  <div>
    <span style="display: inline-block; width: 100px;"><b>Author</b></span>: Deepan Kanagarajan Babu
  </div>
  <div>
    <span style="display: inline-block; width: 100px;"><b>Description</b></span>: In this document, the BLAST-X outputs are analysed. The coxL gene carriers are shortlised and their genes are fetched for further analysis.
  </div>
</div>


<h2 style="font-family: 'Times New Roman'; font-weight: bold;">
    Required Libraries
</h2>

In [1]:
# importing necessary libraries
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

<h2 style="font-family: 'Times New Roman'; font-weight: bold;">
    Required Functions
</h2>

In [3]:
# Defining single function to open multiple file formats (.csv, .txt, .fasta or .fastq (to display only 1st 100 lines), and .xlsx)
def open_file(file_path):
    if file_path.endswith('.csv'):
        # Read CSV file
        data = pd.read_csv(file_path)
        return data
    elif file_path.endswith('.txt'):
        # Read TXT file
        try:
            with open(file_path, 'r') as file:
                data = file.read()
            return data
        except FileNotFoundError:
            raise FileNotFoundError(f"The file {file_path} does not exist.")
    elif file_path.endswith(('.fasta', '.fastq')):
        # Read FASTA or FASTQ file
        try:
            with open(file_path, 'r') as file:
                data = []
                for _ in range(100):
                    line = file.readline()
                    if not line:
                        break
                    data.append(line.strip())
            return data
        except FileNotFoundError:
            raise FileNotFoundError(f"The file {file_path} does not exist.")
    elif file_path.endswith('.xlsx'):
        # Read XLSX file
        try:
            data = pd.read_excel(file_path)
            return data
        except FileNotFoundError:
            raise FileNotFoundError(f"The file {file_path} does not exist.")
    else:
        raise ValueError("Unsupported file format")

In [4]:
# Function to find and remove empty CSV, FASTA, and words files
def empty_files(directory):
    empty_files = []

    # Iterate over each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.csv') or filename.endswith('.docs') or filename.endswith('.fasta'):
            file_path = os.path.join(directory, filename)
            # Check if the file is empty
            if os.path.getsize(file_path) == 0:
                empty_files.append(file_path)
                os.remove(file_path)
            elif filename.endswith('.csv'):
                # Read the CSV file into a dataframe
                df = pd.read_csv(file_path)
                # Check if the dataframe is empty
                if df.empty:
                    empty_files.append(file_path)
                    os.remove(file_path)
    print(f'Done {directory}')

In [5]:
# Function to concate CSV files
def concatenate_csv(input_directory, output_file_path):
    # List to hold dataframes
    dataframes = []

    # Iterate over each CSV file in the directory
    for csv_filename in os.listdir(input_directory):
        if csv_filename.endswith('.csv'):
            csv_file_path = os.path.join(input_directory, csv_filename)
            # Read the CSV file into a dataframe
            df = pd.read_csv(csv_file_path)
            # Add a new column with the input file name
            df['source_file'] = csv_filename
            # Append the dataframe to the list
            dataframes.append(df)

    # Concatenate all dataframes
    concatenated_df = pd.concat(dataframes, ignore_index=True)

    # Save the concatenated dataframe to a new CSV file
    concatenated_df.to_csv(output_file_path, index=False)
    # Leave a confirmation
    print(f"{output_file_path} Successful")

In [6]:
# Function to extract genes from genome when start and end and sequence header are provided
def fetch_genes(df, target_folder, output_folder):
    for target_file in df['target_file'].unique():
        target_df = df[df['target_file'] == target_file]
        target_path = os.path.join(target_folder, target_file)
        
        # Read the genome sequences from the target file
        genome_seqs = {record.id: str(record.seq) for record in SeqIO.parse(target_path, "fasta")}
        
        output_records = []
        query_index = 1  # Initialize query index for each target file
        
        for _, row in target_df.iterrows():
            qseqid = row['qseqid']
            qstart = row['qstart']
            qend = row['qend']
            sseqid = row['sseqid']
            
            # Check if the genome sequence was found
            genome_seq = genome_seqs.get(qseqid, "")
            if not genome_seq:
                print(f"Sequence with ID {qseqid} not found in {target_file}")
                continue
            
            # Extract the gene sequence based on qstart and qend
            if qstart > len(genome_seq) or qend > len(genome_seq):
                print(f"Invalid positions for {qseqid} in {target_file}: qstart={qstart}, qend={qend}, length={len(genome_seq)}")
                continue
            
            if qend < qstart:
                gene_seq = genome_seq[qend-1:qstart][::-1]  # Reverse the sequence
            else:
                gene_seq = genome_seq[qstart-1:qend]
            
            # Check if the extracted gene sequence is empty
            if not gene_seq:
                print(f"Empty gene sequence for {qseqid} in {target_file}: qstart={qstart}, qend={qend}")
                continue
            
            # Create a new sequence record with modified header and description
            seq_record = SeqRecord(
                Seq(gene_seq), 
                id=f"{qseqid} Query{query_index} qstart:{qstart} qend:{qend}", 
                description=sseqid
            )
            output_records.append(seq_record)
            query_index += 1  # Increment query index
        
        # Save all sequence records to a single .fasta file in the output folder
        output_file = os.path.join(output_folder, target_file)
        SeqIO.write(output_records, output_file, "fasta")
        print(f"Saved {output_file}")

<h2 style="font-family: 'Times New Roman'; font-weight: bold;">
    Required Directories Path
</h2>


In [8]:
# Specify the input folder paths
## Path to Metagenomic Assembled Genomes
mags = 'MAG'

## Form 1 CoxL BLAST results
### CSV Output folder
coxl1_csv_results = "CoxL1_Results/csv"

### Words Output folder
coxl1_docs_results = "CoxL1_Results/docs"

### FASTA Output folder
coxl1_fasta_results = "CoxL1_Results/fasta"

## Form 2 CoxL BLAST results
### CSV Output folder
coxl2_csv_results = "CoxL2_Results/csv"

### Words Output folder
coxl2_docs_results = "CoxL2_Results/docs"

### FASTA Output folder
coxl2_fasta_results = "CoxL2_Results/fasta"

# Output Directories
## Directory to store concatenated BLAST CSV for MOTIF form 1
motif_1_csv = "CoxL1_Results/MOTIF"
## Ensure the output directory exists
os.makedirs(motif_1_csv, exist_ok=True)

## File Path to store concatenated BLAST CSV for MOTIF form 1
motif_1_csv_file_path = "CoxL1_Results/MOTIF/coxl1_csv.csv"

## Directory to store concatenated BLAST CSV for MOTIF form 2
motif_2_csv_file_path = "CoxL2_Results/MOTIF"
## Ensure the output directory exists
os.makedirs(motif_2_csv_file_path, exist_ok=True)

## File Path to store concatenated BLAST CSV for MOTIF form 2
motif_2_csv_file_path = "CoxL2_Results/MOTIF/coxl2_csv.csv"

## Directory to store unaligned sequences for MOTIF form 1
motif_1_fasta_path = "CoxL1_Results/MOTIF/fasta"
## Ensure the output directory exists
os.makedirs(motif_1_fasta_path, exist_ok=True)

## Directory to store unaligned sequences for MOTIF form 2
motif_2_fasta_path = "CoxL2_Results/MOTIF/fasta"
## Ensure the output directory exists
os.makedirs(motif_2_fasta_path, exist_ok=True)

<h2 style="font-family: 'Times New Roman'; font-weight: bold;">
    Preprocessing
</h2>

<h3 style="font-family: 'Times New Roman'; font-weight: bold;">
    Remove Empty BLAST Output Files
</h3>

In [11]:
# Using the 'empty_files' function defined above, removing empty output files of MOTIF form 1 results
## Removing empyt CSV files
empty_files(coxl1_csv_results)

## Removing empty FASTA files
empty_files(coxl1_fasta_results)

## Removing empty Words files
empty_files(coxl1_docs_results)

Done CoxL1_Results/csv
Done CoxL1_Results/fasta
Done CoxL1_Results/docs


In [12]:
# Using the 'empty_files' function defined above, removing empty output files of MOTIF form 2 results
## Removing empyt CSV files
empty_files(coxl2_csv_results)

## Removing empty FASTA files
empty_files(coxl2_fasta_results)

## Removing empty Words files
empty_files(coxl2_docs_results)

Done CoxL2_Results/csv
Done CoxL2_Results/fasta
Done CoxL2_Results/docs


<h3 style="font-family: 'Times New Roman'; font-weight: bold;">
    Combine BLAST CSV Outputs
</h3>

In [14]:
# Concatenate MOTIF 1 CSV results
concatenate_csv(coxl1_csv_results, motif_1_csv_file_path)

CoxL1_Results/MOTIF/coxl1_csv.csv Successful


In [15]:
# Concatenate MOTIF 2 CSV results
concatenate_csv(coxl2_csv_results, motif_2_csv_file_path)

CoxL2_Results/MOTIF/coxl2_csv.csv Successful


<h3 style="font-family: 'Times New Roman'; font-weight: bold;">
    Fetch genes from MAGs for MOTIF 1 BLAST Results
</h3>

In [17]:
# Open the MOTIF 1 concatenated CSV file
coxl1_csv = open_file(motif_1_csv_file_path)
coxl1_csv.head()

,qseqid,sseqid,pident,qcovs,evalue,bitscore,qstart,qend,sstart,send,length,source_file
0,Genome20_S3Ck127_316360,Mycobacterium_marinum_M,24.634,42,5.050000e-43,163.0,4980,2872,53,784,751,Bdellovibrionaceae_bacterium_ERS6626579_scores...
1,Genome20_S3Ck127_316360,Mycobacterium_tusciae_WP_006247553,24.433,42,2.450000e-41,158.0,4980,2872,53,784,749,Bdellovibrionaceae_bacterium_ERS6626579_scores...
2,Genome20_S3Ck127_316360,Rhodococcus_opacus_B4,24.074,39,4.120000e-41,157.0,4980,3022,56,735,702,Bdellovibrionaceae_bacterium_ERS6626579_scores...
3,Genome20_S3Ck127_316360,Micromonospora_aurantiaca_ATCC_27029,24.536,42,4.150000e-41,157.0,4980,2872,53,784,754,Bdellovibrionaceae_bacterium_ERS6626579_scores...
4,Genome20_S3Ck127_316360,Mycobacterium_ulcerans_Agy99_YP_904347,24.528,42,5.750000e-41,156.0,4953,2872,62,784,742,Bdellovibrionaceae_bacterium_ERS6626579_scores...


In [18]:
# Check the format of source_file
coxl1_csv['source_file'].unique()[0]

'Bdellovibrionaceae_bacterium_ERS6626579_scores.csv'

In [19]:
# Create a target_file column, which is the file name of genomes, from the source file 
coxl1_csv['target_file'] = coxl1_csv['source_file'].apply(lambda x: x[:-11] + '.fasta')
coxl1_csv

,qseqid,sseqid,pident,qcovs,evalue,bitscore,qstart,qend,sstart,send,length,source_file,target_file
0,Genome20_S3Ck127_316360,Mycobacterium_marinum_M,24.634,42,5.050000e-43,163.0,4980,2872,53,784,751,Bdellovibrionaceae_bacterium_ERS6626579_scores...,Bdellovibrionaceae_bacterium_ERS6626579.fasta
1,Genome20_S3Ck127_316360,Mycobacterium_tusciae_WP_006247553,24.433,42,2.450000e-41,158.0,4980,2872,53,784,749,Bdellovibrionaceae_bacterium_ERS6626579_scores...,Bdellovibrionaceae_bacterium_ERS6626579.fasta
2,Genome20_S3Ck127_316360,Rhodococcus_opacus_B4,24.074,39,4.120000e-41,157.0,4980,3022,56,735,702,Bdellovibrionaceae_bacterium_ERS6626579_scores...,Bdellovibrionaceae_bacterium_ERS6626579.fasta
3,Genome20_S3Ck127_316360,Micromonospora_aurantiaca_ATCC_27029,24.536,42,4.150000e-41,157.0,4980,2872,53,784,754,Bdellovibrionaceae_bacterium_ERS6626579_scores...,Bdellovibrionaceae_bacterium_ERS6626579.fasta
4,Genome20_S3Ck127_316360,Mycobacterium_ulcerans_Agy99_YP_904347,24.528,42,5.750000e-41,156.0,4953,2872,62,784,742,Bdellovibrionaceae_bacterium_ERS6626579_scores...,Bdellovibrionaceae_bacterium_ERS6626579.fasta
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99069,Genome526_k127_368022,Solirubrobacter_sp._URHD008_2,24.074,12,2.330000e-15,75.9,2280,3665,272,738,486,uncultured_Xylophilus_sp._ERS6627302_scores.csv,uncultured_Xylophilus_sp._ERS6627302.fasta
99070,Genome526_k127_368022,Pseduonocardia_autotrophica_1,27.444,7,1.240000e-14,73.6,2880,3665,486,740,266,uncultured_Xylophilus_sp._ERS6627302_scores.csv,uncultured_Xylophilus_sp._ERS6627302.fasta
99071,Genome526_k127_368022,Salinarimonas_rosea_DSM_21201_2523196126,21.942,13,7.830000e-14,70.9,2283,3695,291,770,515,uncultured_Xylophilus_sp._ERS6627302_scores.csv,uncultured_Xylophilus_sp._ERS6627302.fasta
99072,Genome526_k127_368022,Nocardioides_sp._JS614_2,27.106,7,9.760000e-14,70.5,2862,3665,491,749,273,uncultured_Xylophilus_sp._ERS6627302_scores.csv,uncultured_Xylophilus_sp._ERS6627302.fasta


In [20]:
# Save the updated CoxL1 CSV file
coxl1_csv.to_csv(f'{motif_1_csv_file_path}', index=False)

In [21]:
# Fetch the matches from MOTIF 1 BLAST
fetch_genes(coxl1_csv, mags, motif_1_fasta_path)

Saved CoxL1_Results/MOTIF/fasta\Bdellovibrionaceae_bacterium_ERS6626579.fasta
Saved CoxL1_Results/MOTIF/fasta\Bdellovibrionaceae_bacterium_ERS6626826.fasta
Saved CoxL1_Results/MOTIF/fasta\Burkholderiaceae_bacterium_ERS6626828.fasta
Saved CoxL1_Results/MOTIF/fasta\Burkholderiaceae_bacterium_ERS6626861.fasta
Saved CoxL1_Results/MOTIF/fasta\Candidatus_Sericytochromatia_bacterium_ERS6626968.fasta
Saved CoxL1_Results/MOTIF/fasta\Rhizobiaceae_bacterium_ERS6626796.fasta
Saved CoxL1_Results/MOTIF/fasta\uncultured_Achromobacter_sp._ERS6626913.fasta
Saved CoxL1_Results/MOTIF/fasta\uncultured_Acidovorax_sp._ERS6626561.fasta
Saved CoxL1_Results/MOTIF/fasta\uncultured_Acidovorax_sp._ERS6626565.fasta
Saved CoxL1_Results/MOTIF/fasta\uncultured_Acidovorax_sp._ERS6626576.fasta
Saved CoxL1_Results/MOTIF/fasta\uncultured_Acidovorax_sp._ERS6626591.fasta
Saved CoxL1_Results/MOTIF/fasta\uncultured_Acidovorax_sp._ERS6626592.fasta
Saved CoxL1_Results/MOTIF/fasta\uncultured_Acidovorax_sp._ERS6626596.fasta
Save

<h3 style="font-family: 'Times New Roman'; font-weight: bold;">
    Fetch genes from MAGs for MOTIF 2 BLAST Results
</h3>

In [23]:
# Open the MOTIF 2 concatenated results
coxl2_csv = open_file(motif_2_csv_file_path)
coxl2_csv.head()

,qseqid,sseqid,pident,qcovs,evalue,bitscore,qstart,qend,sstart,send,length,source_file
0,Genome20_S3Ck127_316360,BAB48572_Form2_Rhodopseudomonas_palustris_BisA...,24.931,42,8.800000e-31,119.0,4944,2872,52,754,726,Bdellovibrionaceae_bacterium_ERS6626579_scores...
1,Genome20_S3Ck127_316360,BAB48572_Form2_Mesorhizobium_japonicum_MAFF,24.933,41,1.080000e-26,105.0,4953,2911,48,760,742,Bdellovibrionaceae_bacterium_ERS6626579_scores...
2,Genome267_k127_201050,BAB48572_Form2_Mesorhizobium_japonicum_MAFF,23.846,21,4.900000e-35,134.0,6807,8963,7,767,780,Bdellovibrionaceae_bacterium_ERS6626826_scores...
3,Genome267_k127_201050,BAB48572_Form2_Rhodopseudomonas_palustris_BisA...,24.791,17,4.110000e-34,130.0,7299,9014,188,766,597,Bdellovibrionaceae_bacterium_ERS6626826_scores...
4,Genome267_k127_433915,BAB48572_Form2_Rhodopseudomonas_palustris_BisA...,24.156,20,1.980000e-34,132.0,3178,995,8,741,770,Bdellovibrionaceae_bacterium_ERS6626826_scores...


In [30]:
# Create a target_file column, which the file name of genomes, from the source file 
coxl2_csv['target_file'] = coxl2_csv['source_file'].apply(lambda x: x[:-11] + '.fasta')
coxl2_csv

,qseqid,sseqid,pident,qcovs,evalue,bitscore,qstart,qend,sstart,send,length,source_file,target_file
0,Genome20_S3Ck127_316360,BAB48572_Form2_Rhodopseudomonas_palustris_BisA...,24.931,42,8.800000e-31,119.0,4944,2872,52,754,726,Bdellovibrionaceae_bacterium_ERS6626579_scores...,Bdellovibrionaceae_bacterium_ERS6626579.fasta
1,Genome20_S3Ck127_316360,BAB48572_Form2_Mesorhizobium_japonicum_MAFF,24.933,41,1.080000e-26,105.0,4953,2911,48,760,742,Bdellovibrionaceae_bacterium_ERS6626579_scores...,Bdellovibrionaceae_bacterium_ERS6626579.fasta
2,Genome267_k127_201050,BAB48572_Form2_Mesorhizobium_japonicum_MAFF,23.846,21,4.900000e-35,134.0,6807,8963,7,767,780,Bdellovibrionaceae_bacterium_ERS6626826_scores...,Bdellovibrionaceae_bacterium_ERS6626826.fasta
3,Genome267_k127_201050,BAB48572_Form2_Rhodopseudomonas_palustris_BisA...,24.791,17,4.110000e-34,130.0,7299,9014,188,766,597,Bdellovibrionaceae_bacterium_ERS6626826_scores...,Bdellovibrionaceae_bacterium_ERS6626826.fasta
4,Genome267_k127_433915,BAB48572_Form2_Rhodopseudomonas_palustris_BisA...,24.156,20,1.980000e-34,132.0,3178,995,8,741,770,Bdellovibrionaceae_bacterium_ERS6626826_scores...,Bdellovibrionaceae_bacterium_ERS6626826.fasta
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2194,Genome526_k127_12441,BAB48572_Form2_Rhodopseudomonas_palustris_BisA...,31.604,1,4.330000e-07,45.1,1502,903,551,754,212,uncultured_Xylophilus_sp._ERS6627302_scores.csv,uncultured_Xylophilus_sp._ERS6627302.fasta
2195,Genome526_k127_152992,BAB48572_Form2_Rhodopseudomonas_palustris_BisA...,25.142,3,2.390000e-24,102.0,51673,53688,9,688,704,uncultured_Xylophilus_sp._ERS6627302_scores.csv,uncultured_Xylophilus_sp._ERS6627302.fasta
2196,Genome526_k127_152992,BAB48572_Form2_Mesorhizobium_japonicum_MAFF,23.077,3,2.420000e-17,79.3,51673,53721,7,722,741,uncultured_Xylophilus_sp._ERS6627302_scores.csv,uncultured_Xylophilus_sp._ERS6627302.fasta
2197,Genome526_k127_368022,BAB48572_Form2_Mesorhizobium_japonicum_MAFF,25.146,13,2.320000e-25,102.0,2199,3680,252,752,513,uncultured_Xylophilus_sp._ERS6627302_scores.csv,uncultured_Xylophilus_sp._ERS6627302.fasta


In [31]:
# Save the updated CoxL2 CSV file
coxl2_csv.to_csv(f'{motif_2_csv_file_path}', index=False)

In [32]:
# Fetch the matches from MOTIF 2 BLAST
fetch_genes(coxl2_csv, mags, motif_2_fasta_path)

Saved CoxL2_Results/MOTIF/fasta\Bdellovibrionaceae_bacterium_ERS6626579.fasta
Saved CoxL2_Results/MOTIF/fasta\Bdellovibrionaceae_bacterium_ERS6626826.fasta
Saved CoxL2_Results/MOTIF/fasta\Burkholderiaceae_bacterium_ERS6626828.fasta
Saved CoxL2_Results/MOTIF/fasta\Burkholderiaceae_bacterium_ERS6626861.fasta
Saved CoxL2_Results/MOTIF/fasta\Candidatus_Sericytochromatia_bacterium_ERS6626968.fasta
Saved CoxL2_Results/MOTIF/fasta\Rhizobiaceae_bacterium_ERS6626796.fasta
Saved CoxL2_Results/MOTIF/fasta\uncultured_Achromobacter_sp._ERS6626913.fasta
Saved CoxL2_Results/MOTIF/fasta\uncultured_Acidovorax_sp._ERS6626561.fasta
Saved CoxL2_Results/MOTIF/fasta\uncultured_Acidovorax_sp._ERS6626565.fasta
Saved CoxL2_Results/MOTIF/fasta\uncultured_Acidovorax_sp._ERS6626576.fasta
Saved CoxL2_Results/MOTIF/fasta\uncultured_Acidovorax_sp._ERS6626591.fasta
Saved CoxL2_Results/MOTIF/fasta\uncultured_Acidovorax_sp._ERS6626592.fasta
Saved CoxL2_Results/MOTIF/fasta\uncultured_Acidovorax_sp._ERS6626596.fasta
Save